In [1]:
import numpy as np
import torch
from ucimlrepo import fetch_ucirepo 
import torch.nn as nn     
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import main
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 
y["class"]=y["class"].replace("Iris-setosa",0.0)
y["class"]=y["class"].replace("Iris-versicolor",1.0)
y["class"]=y["class"].replace("Iris-virginica",2.0)
# metadata 
X=X.values
y=y.values
y=y.squeeze(axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
mean = X_train.mean(axis=0)  # Mean of training set
std = X_train.std(axis=0)  # Standard deviation of training set

# Avoid division by zero for constant features
std[std == 0] = 1

# Apply mean normalization (centering and scaling) to training and test sets
X_train_numpy = (X_train - mean) / std
X_test_numpy = (X_test - mean) / std

X_train=torch.FloatTensor(X_train_numpy)
X_test=torch.FloatTensor(X_test_numpy)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

C:\Users\Sebas\AppData\Local\Temp\ipykernel_10528\167960093.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["class"]=y["class"].replace("Iris-setosa",0.0)
C:\Users\Sebas\AppData\Local\Temp\ipykernel_10528\167960093.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["class"]=y["class"].replace("Iris-versicolor",1.0)
C:\Users\Sebas\AppData\Local\Temp\ipykernel_10528\167960093.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the

In [2]:
### Version one with hard thresholding
class Model(nn.Module):
    def __init__(self,in_features=4,h1=16,out=3):
        super().__init__()
        self.fc1=nn.Linear(in_features,h1,bias=False)
        self.out=nn.Linear(h1,out)
    def forward(self,x):
        hidden_activation = F.relu(self.fc1(x))  # Hidden layer with ReLU activation
        output = self.out(hidden_activation)  # Final output
        return output, hidden_activation
def iris_training(h1,alpha):
    model=Model(h1=h1)
    W_untrained=copy.deepcopy(next(model.parameters())).detach().numpy().astype(float)
    #print(W_untrained)
    criterion=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
    epochs=100
    losses=[]
    for i in range(epochs):
        y_pred,hidden_activation=model.forward(X_train)
        classification_loss=loss=criterion(y_pred,y_train)
        neuron_activity_loss = -torch.mean(F.relu(hidden_activation))
        loss=classification_loss+alpha*neuron_activity_loss
        losses.append(classification_loss.detach().numpy())
        #if i % 10 == 0:
           # print(f'Epoch: {i} and loss: {loss}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    W_trained=next(model.parameters()).detach().numpy().astype(float)
    return W_untrained,W_trained,classification_loss.detach().numpy()
#plt.plot(range(epochs),losses)


In [ ]:
total_untrained=[]
total_trained=[]
total_losses=[]
total_train_losses=[]
total_test_losses=[]
for m in range(14,35):
    print(f"start {m}")
    train_losses=[]
    untrained=[]
    trained=[]
    for i in range(500):
        res=iris_training(m,0)
        untrained.append(res[0])
        trained.append(res[1])
        train_losses.append(res[2])

    train_losses=np.array(train_losses)

    total_train_losses.append(np.mean(train_losses))

    print(f"train loss= {np.mean(train_losses)} ")
    res=0


    for W in untrained:
    # print(W.T.shape)
        if not main.findCell(np.array([]),W):
            res+=2

    train=0

    for W in trained:

        non_inj_count=0
        if not main.findCell(np.array([]), W):
            train+=2


   
    total_untrained.append(res)
    total_trained.append(train)

    print(f"m={m} untrained= {res},trained = {train}" )
print(total_untrained,total_trained,total_train_losses,total_test_losses)


start 14
train loss= 0.161274254322052 
m=14 untrained= 10,trained = 4
start 15
train loss= 0.1593208909034729 
m=15 untrained= 62,trained = 16
start 16
train loss= 0.15589338541030884 
m=16 untrained= 110,trained = 48
start 17
train loss= 0.15490448474884033 
m=17 untrained= 210,trained = 102
start 18
train loss= 0.15356218814849854 
m=18 untrained= 308,trained = 162
start 19
train loss= 0.15076622366905212 
m=19 untrained= 490,trained = 286
start 20
train loss= 0.15090371668338776 


Exception ignored in: <function Cplex.__del__ at 0x0000025F450970A0>
Traceback (most recent call last):
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\__init__.py", line 925, in __del__
    self.end()
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\__init__.py", line 921, in end
    self._env._end()
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\_internal\__init__.py", line 153, in _end
    self._delete_stream(chnl_idx)
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\_internal\__init__.py", line 248, in _delete_stream
    self._streams[which_channel]._end()
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\_internal\_ostream.py", line 88, in _end
    self.flush()
  File "c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\cplex\_internal\_ostream.py", line 148, in flush
   

In [3]:
for m in range(14,35):
    print(f"start {m}")
    losses=[]
    untrained=[]
    trained=[]
    for i in range(500):
        res=iris_training(m,-0.1)
        losses.append(res[2])
        untrained.append(res[0])
        trained.append(res[1])
    losses=np.array(losses)
    print(np.mean(losses))
    res=0
    for W in untrained:
    # print(W.T.shape)
        if not main.findCell(np.array([]),W):
            res+=2
    res
    train=0
    for W in trained:
        if not main.findCell(np.array([]), W):
            train+=2
    print(f"m={m} untrained= {res},trained = {train}")


start 14


c:\Users\Sebas\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.17846596
m=14 untrained= 6,trained = 0
start 15
0.17654671
m=15 untrained= 50,trained = 2
start 16
0.17436498
m=16 untrained= 124,trained = 14
start 17
0.17363718
m=17 untrained= 228,trained = 20
start 18
0.17162469
m=18 untrained= 382,trained = 48
start 19
0.17055106
m=19 untrained= 466,trained = 60
start 20
0.16940644
m=20 untrained= 608,trained = 116
start 21
0.16765751
m=21 untrained= 668,trained = 150
start 22
0.16799492
m=22 untrained= 752,trained = 174
start 23
0.16624339
m=23 untrained= 820,trained = 244
start 24
0.16547903
m=24 untrained= 872,trained = 320
start 25
0.16449836
m=25 untrained= 910,trained = 384
start 26
0.16381554
m=26 untrained= 958,trained = 412
start 27
0.16302942
m=27 untrained= 956,trained = 522
start 28
0.16228731
m=28 untrained= 970,trained = 580
start 29
0.16141848
m=29 untrained= 982,trained = 610
start 30
0.1607486
m=30 untrained= 978,trained = 660
start 31
0.16008154
m=31 untrained= 992,trained = 722
start 32
0.15940036
m=32 untrained= 992,trained =